# Scenario Comparison: Capacity Boost vs. Bus Intervention

This notebook compares two intervention scenarios against the current (baseline) Milwaukee network:
- **Scenario 1:** Increase capacity by 20% on all links.
- **Scenario 2:** Add a bus line on the most congested path (mode shift of a share of trips on the top-demand OD).

We reuse the datasets in `../data` and mirror the style of the analyses in the course TPs (summary stats, congestion indicators, and comparative charts).

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 80)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries ready.")

In [ ]:
# Load core datasets
nodes_df = pd.read_csv('../data/node.csv')
links_df = pd.read_csv('../data/link.csv')
demand_df = pd.read_csv('../data/demand.csv')

# Optional: link performance (if available)
try:
    link_performance = pd.read_csv('../data/link_performance.csv')
    print("Loaded link_performance.csv")
except FileNotFoundError:
    link_performance = None
    print("link_performance.csv not found; will synthesize baseline volumes")

print(f"Nodes: {len(nodes_df):,} | Links: {len(links_df):,} | OD pairs: {len(demand_df):,}")
print(f"Total demand (trips): {demand_df['volume'].sum():,.0f}")

In [ ]:
# Helper functions

def build_graph(links: pd.DataFrame) -> tuple[nx.DiGraph, dict]:
    G = nx.DiGraph()
    for _, r in links.iterrows():
        G.add_edge(r['from_node_id'], r['to_node_id'], length=r.get('length', 1.0))
    edge_to_link = {(r['from_node_id'], r['to_node_id']): r['link_id'] for _, r in links.iterrows()}
    return G, edge_to_link


def ensure_link_perf(lp: pd.DataFrame | None, links: pd.DataFrame) -> pd.DataFrame:
    if lp is None:
        df = links[['link_id', 'from_node_id', 'to_node_id', 'capacity']].copy()
        df['volume'] = df['capacity'] * 0.6  # synthetic baseline volume
    else:
        df = lp.copy()
        if 'capacity' not in df.columns and 'link_capacity' in df.columns:
            df = df.rename(columns={'link_capacity': 'capacity'})
        if 'volume' not in df.columns:
            # choose the best available volume-like column
            for cand in ['mod_vol_auto', 'ref_volume', 'base_demand_volume', 'obs_volume']:
                if cand in df.columns:
                    df['volume'] = df[cand]
                    break
            if 'volume' not in df.columns:
                df['volume'] = df['capacity'] * 0.6
        df = df[['link_id', 'from_node_id', 'to_node_id', 'capacity', 'volume']].copy()
    df['vc_ratio'] = df['volume'] / df['capacity'].replace(0, np.nan)
    return df


def summarize_network(name: str, link_perf: pd.DataFrame, demand: pd.DataFrame, links: pd.DataFrame) -> dict:
    metrics = {
        'Scenario': name,
        'Total Demand (trips)': demand['volume'].sum(),
        'Avg OD Volume (trips)': demand['volume'].mean(),
        'Max OD Volume (trips)': demand['volume'].max(),
        'Total Capacity (veh/h)': links['capacity'].sum(),
        'Mean V/C': link_perf['vc_ratio'].mean(),
        'Median V/C': link_perf['vc_ratio'].median(),
        'Pct Links V/C>1.0': (link_perf['vc_ratio'] > 1.0).mean() * 100,
        'Max V/C': link_perf['vc_ratio'].max(),
    }
    return metrics


def shortest_path_for_top_od(G: nx.DiGraph, demand: pd.DataFrame) -> tuple[pd.Series, list[int]]:
    top_od = demand.nlargest(1, 'volume').iloc[0]
    try:
        path = nx.shortest_path(G, source=top_od['o_zone_id'], target=top_od['d_zone_id'], weight='length')
    except nx.NetworkXNoPath:
        path = []
    return top_od, path


def path_edges(path_nodes: list[int]) -> list[tuple[int, int]]:
    if len(path_nodes) < 2:
        return []
    return list(zip(path_nodes[:-1], path_nodes[1:]))


def reduce_link_volumes(link_perf: pd.DataFrame, edges: list[tuple[int, int]], edge_lookup: dict, reduction: float) -> pd.DataFrame:
    df = link_perf.copy()
    if reduction <= 0 or not edges:
        return df
    per_edge = reduction / max(len(edges), 1)
    for u, v in edges:
        link_id = edge_lookup.get((u, v))
        if link_id is None:
            continue
        mask = df['link_id'] == link_id
        df.loc[mask, 'volume'] = np.maximum(df.loc[mask, 'volume'] - per_edge, 0)
    df['vc_ratio'] = df['volume'] / df['capacity'].replace(0, np.nan)
    return df

In [ ]:
# Build graph and identify congested path (top-demand OD as proxy)
G, edge_lookup = build_graph(links_df)
top_od, path_nodes = shortest_path_for_top_od(G, demand_df)
edges_on_path = path_edges(path_nodes)

print("Top-demand OD (proxy for congested corridor):")
print(top_od[['o_zone_id', 'd_zone_id', 'volume']])
print(f"Path nodes: {path_nodes}")
print(f"Edges on path: {edges_on_path}")

# Baseline link performance
lp_base = ensure_link_perf(link_performance, links_df)

# Scenario 1: +20% capacity on all links
links_s1 = links_df.copy()
links_s1['capacity'] *= 1.20
lp_s1 = lp_base.copy()
lp_s1['capacity'] *= 1.20
lp_s1['vc_ratio'] = lp_s1['volume'] / lp_s1['capacity'].replace(0, np.nan)

demand_s1 = demand_df.copy()  # unchanged demand

# Scenario 2: Bus line diverts share of trips on congested path
SHIFT_SHARE = 0.20
links_s2 = links_df.copy()  # capacity unchanged

demand_s2 = demand_df.copy()
top_idx = top_od.name
demand_s2.loc[top_idx, 'volume'] *= (1 - SHIFT_SHARE)
bus_diverted = top_od['volume'] * SHIFT_SHARE

lp_s2 = reduce_link_volumes(lp_base, edges_on_path, edge_lookup, bus_diverted)

# Collect metrics
rows = []
rows.append(summarize_network('Baseline', lp_base, demand_df, links_df))
rows.append(summarize_network('Cap +20%', lp_s1, demand_s1, links_s1))
rows.append(summarize_network('Bus on congested path', lp_s2, demand_s2, links_s2))

metrics_df = pd.DataFrame(rows)
metrics_df

In [ ]:
# Plot comparative metrics
plot_cols = [
    'Total Demand (trips)',
    'Total Capacity (veh/h)',
    'Mean V/C',
    'Pct Links V/C>1.0',
    'Max V/C',
]

plot_df = metrics_df.set_index('Scenario')[plot_cols].reset_index()
plot_df_melt = plot_df.melt(id_vars='Scenario', var_name='Metric', value_name='Value')

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=plot_df_melt, x='Metric', y='Value', hue='Scenario', ax=ax)
ax.set_title('Scenario Comparison')
ax.set_ylabel('Value')
ax.set_xlabel('Metric')
ax.tick_params(axis='x', rotation=25)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("Key notes:")
print(f"- Bus diversion share: {SHIFT_SHARE*100:.0f}% of top OD (volume {top_od['volume']:,.0f} trips)")
print(f"- Diverted trips: {bus_diverted:,.0f}")
print(f"- Path edges affected: {len(edges_on_path)}")

## How to run
1) Run the import and data-loading cells (Cells 2-3).
2) Run the helper and scenario cells (Cells 4-6) to build metrics.
3) Run the plotting cell (Cell 7) for the comparison chart.

## Notes and assumptions
- The “most congested path” is approximated by the shortest path of the top-demand OD pair. If you prefer to use the highest `vc_ratio` link corridor, swap the selection logic in the `shortest_path_for_top_od` helper.
- If `link_performance.csv` is missing, synthetic volumes at 60% of capacity are used to compute V/C.
- The bus intervention diverts 20% of that OD’s trips and subtracts them evenly across links along the path. Adjust `SHIFT_SHARE` if you want a different mode shift assumption.
- If you already computed richer results in `antoine analysis network.ipynb`, you can import them (e.g., pre-built GeoDataFrames) to refine path selection or add spatial plots.